In [20]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import time

device = torch.device('mps')

In [21]:
# indirince bunu bi denerik
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [22]:


all_transforms = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])
# Create Training dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=all_transforms, download=True)

# Create Testing dataset
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=all_transforms, download=True)

batch_size = 128
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Files already downloaded and verified


In [44]:
# Define the CNN model
class ConvNeuralNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()
        self.drop1 = nn.Dropout(0.25)
        self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
 
        self.fc1 = nn.Linear(16384, 128)
        self.act5 = nn.ReLU()
        self.drop2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        out = self.act1(self.conv_layer1(x))
        out = self.act2(self.conv_layer2(out))
        out = self.drop1(out)
        out = self.max_pool1(out)
        
        out = out.view(out.size(0), -1)
        out = self.drop2(self.act5(self.fc1(out)))
        out = self.fc2(out)
        return out

In [45]:
num_classes = 10
learning_rate = 0.001
num_epochs = 20


model = ConvNeuralNet(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    loss = None
    t = time.time()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    correct = 0
    with torch.no_grad():
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {}, Elapsed: {} '.format(epoch + 1, num_epochs, loss.item(),
                                                                           100 * correct / total, time.time() - t))

Epoch [1/20], Loss: 1.4866, Accuracy: 42.2, Elapsed: 18.286622047424316 
Epoch [2/20], Loss: 1.3297, Accuracy: 50.74, Elapsed: 18.17188787460327 
Epoch [3/20], Loss: 1.2245, Accuracy: 52.98, Elapsed: 18.082685232162476 
Epoch [4/20], Loss: 1.2724, Accuracy: 55.3, Elapsed: 18.015759229660034 
Epoch [5/20], Loss: 1.2128, Accuracy: 57.46, Elapsed: 18.01895809173584 
Epoch [6/20], Loss: 1.2538, Accuracy: 58.74, Elapsed: 18.005617141723633 
Epoch [7/20], Loss: 1.3215, Accuracy: 58.94, Elapsed: 18.008980989456177 
Epoch [8/20], Loss: 0.9250, Accuracy: 58.86, Elapsed: 18.04648494720459 
Epoch [9/20], Loss: 0.7841, Accuracy: 60.36, Elapsed: 17.984088897705078 
Epoch [10/20], Loss: 1.0321, Accuracy: 60.64, Elapsed: 18.09227991104126 
Epoch [11/20], Loss: 1.1158, Accuracy: 60.7, Elapsed: 18.091243028640747 
Epoch [12/20], Loss: 0.7492, Accuracy: 60.52, Elapsed: 18.11512589454651 
Epoch [13/20], Loss: 0.9666, Accuracy: 60.72, Elapsed: 18.11518883705139 
Epoch [14/20], Loss: 1.0016, Accuracy: 60.9

In [26]:
Epoch [1/20], Loss: 1.3613, Accuracy: 53.62, Elapsed: 15.32989501953125 
Epoch [2/20], Loss: 0.8447, Accuracy: 63.2, Elapsed: 15.123994827270508 
Epoch [3/20], Loss: 0.8678, Accuracy: 65.93, Elapsed: 15.185999870300293 
Epoch [4/20], Loss: 0.8962, Accuracy: 68.71, Elapsed: 15.104533195495605 
Epoch [5/20], Loss: 0.5207, Accuracy: 70.58, Elapsed: 15.10148310661316 
Epoch [6/20], Loss: 0.6476, Accuracy: 70.49, Elapsed: 15.253604650497437 
Epoch [7/20], Loss: 0.5595, Accuracy: 70.84, Elapsed: 15.609240055084229 
Epoch [8/20], Loss: 0.5878, Accuracy: 70.2, Elapsed: 15.391446113586426 
Epoch [9/20], Loss: 0.4980, Accuracy: 69.8, Elapsed: 15.125716209411621 
Epoch [10/20], Loss: 0.2939, Accuracy: 69.97, Elapsed: 15.214236974716187 
Epoch [11/20], Loss: 0.3107, Accuracy: 70.41, Elapsed: 15.064754962921143 
Epoch [12/20], Loss: 0.3099, Accuracy: 70.17, Elapsed: 15.302250146865845 
Epoch [13/20], Loss: 0.3888, Accuracy: 68.96, Elapsed: 15.270372152328491 
Epoch [14/20], Loss: 0.3394, Accuracy: 69.54, Elapsed: 15.145703077316284 
Epoch [15/20], Loss: 0.2791, Accuracy: 69.18, Elapsed: 15.143717050552368

SyntaxError: only single target (not tuple) can be annotated (175963.py, line 1)